In [1]:
from sklearn.datasets import fetch_20newsgroups


news = fetch_20newsgroups(subset='all')

In [58]:
print type(news.data)
print type(news.target)
print type(news.target_names)

<type 'list'>
<type 'numpy.ndarray'>
<type 'list'>


In [10]:
print len(news.data)
print len(news.target)
print len(news.target_names)

18846
18846
20


In [63]:
print news.target_names[news.target[0]]
print "Data: " + str(len(news.data[7]))

rec.sport.hockey
Data: 1278


In [27]:
from sklearn.model_selection import train_test_split

def train(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
 
    model.fit(X_train, y_train)
    print "Accuracy: %s" % model.score(X_test, y_test)

In [28]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

### TF.IDF

### TD.IDF OVER EVERYWORD

In [29]:
trial1 = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', MultinomialNB()),
])
train(trial1,news.data,news.target)

Accuracy: 0.8511936339522547
